In [3]:
import pyscf

import numpy as np
import scipy as sp

import math
import itertools

def rudimentary_fci_matrix(h1e, h2e, norb, nelec):

    # We need to generate all possible determinants.
    # Includes both alpha and beta electrons...

    # We can use the itertools library to generate combinations of orbitals.
    # We are not restricting the possible spin yet.

    determinants = list(itertools.combinations(range(2*norb), nelec))
    # This will give us all possible combinations of spinorbitals for the given number of electrons.

    dim = len(determinants)
    # This is the dimension of the FCI space.

    def count_diff(det1, det2):
        return len(det1) - len(list(set(det1).intersection(set(det2))))

    # slater condon rules:
    def slater_condon_rule(det1, det2):

        one_electron_energy = 0.0
        two_electron_energy = 0.0
        # We will calculate the one-electron and two-electron contributions separately.

        diff = count_diff(det1, det2)

        if diff > 2:
            return 0.0

        elif diff == 0:
            # then the two determinants are the same; diagonal element.
            for i in det1:
                one_electron_energy += h1e[i % norb, i % norb]
            for i in det1:
                for j in det1:
                    two_electron_energy += 0.5*h2e[i%norb, i%norb, j%norb, j%norb]
                    if i < norb and j < norb:
                        two_electron_energy -= 0.5*h2e[i%norb, j%norb, j%norb, i%norb]
                    elif i >= norb and j >= norb:
                        two_electron_energy -= 0.5*h2e[i%norb, j%norb, j%norb, i%norb]
            return one_electron_energy + two_electron_energy
        
        elif diff == 1:
            diff_det1 = []
            diff_det2 = []
            nswaps1 = 0
            nswaps2 = 0
            # which elements are different?
            for i in range(nelec):
                # if i not in det2, then need to permute corresponding element to the end.
                if det1[i] not in det2:
                    diff_det1.append(det1[i])
                    nswaps1 += nelec - i - 1
                if det2[i] not in det1:
                    diff_det2.append(det2[i])
                    nswaps2 += nelec - i - 1
            
            nswaps = nswaps1 + nswaps2
            # now we have the two different elements.
            if diff_det1[0] < norb and diff_det2[0] >= norb:
                return 0.0
            elif diff_det1[0] >= norb and diff_det2[0] < norb:
                return 0.0
            else:
            # We can calculate the one-electron contribution.
                one_electron_energy += h1e[diff_det1[0] % norb, diff_det2[0] % norb]
                # and the two-electron contribution.
                for i in range(nelec):
                    if det1[i] != diff_det1[0]:
                        two_electron_energy += h2e[diff_det1[0] % norb, diff_det2[0] % norb, det1[i] % norb, det1[i] % norb]
                        if det1[i] < norb and diff_det1[0] < norb:
                            two_electron_energy -= h2e[diff_det1[0] % norb, det1[i] % norb, det1[i] % norb, diff_det2[0] % norb]
                        elif det1[i] >= norb and diff_det1[0] >= norb:
                            two_electron_energy -= h2e[diff_det1[0] % norb, det1[i] % norb, det1[i] % norb, diff_det2[0] % norb]
            one_electron_energy *= (-1)**nswaps
            two_electron_energy *= (-1)**nswaps
            return one_electron_energy + two_electron_energy

        elif diff == 2:
            diff_det1 = []
            diff_det2 = []
            nswaps1 = 0
            nswaps2 = 0

            for i in range(nelec):
                if det1[i] not in det2:
                    diff_det1.append(det1[i])
                    nswaps1 += nelec - i - 1
                if det2[i] not in det1:
                    diff_det2.append(det2[i])
                    nswaps2 += nelec - i - 1
            
            nswaps = nswaps1 + nswaps2

            if diff_det1[0] < norb and diff_det2[0] >= norb:
                return 0.0
            elif diff_det1[0] >= norb and diff_det2[0] < norb:
                return 0.0
            elif diff_det1[1] < norb and diff_det2[1] >= norb:
                return 0.0
            elif diff_det1[1] >= norb and diff_det2[1] < norb:
                return 0.0
            else:
                # only contribution is going to be from the two-electron term.
                two_electron_energy += h2e[diff_det1[0] % norb, diff_det2[0] % norb, diff_det1[1] % norb, diff_det2[1] % norb]
                if diff_det1[1] < norb and diff_det2[1] < norb:
                    two_electron_energy -= h2e[diff_det1[0] % norb, diff_det2[1] % norb, diff_det2[0] % norb, diff_det1[1] % norb]
                elif diff_det1[0] >= norb and diff_det2[0] >= norb:
                    two_electron_energy -= h2e[diff_det1[0] % norb, diff_det2[1] % norb, diff_det2[0] % norb, diff_det1[1] % norb]
            one_electron_energy = 0.0
            two_electron_energy *= (-1)**nswaps
            return one_electron_energy + two_electron_energy
    
    # Now we can construct the Hamiltonian matrix.
    hamiltonian = np.zeros((dim, dim))

    for i in range(dim):
        for j in range(dim):
            hamiltonian[i, j] = slater_condon_rule(determinants[i], determinants[j])

    # symmetrize
    hamiltonian = 0.5 * (hamiltonian + hamiltonian.T)
    
    return hamiltonian

In [4]:
hamiltonian = rudimentary_fci_matrix(
    h1e=np.load("h1e.npy"),
    h2e=np.load("h2e.npy"),
    norb=6,
    nelec=6
)

In [5]:
def rudimentaryDavidson(matrix):

    # create a guess matrix
    dimension = len(matrix)
    guessV1 = np.zeros(dimension)
    guessV2 = np.zeros(dimension)
    for i in range(dimension):
        guessV1[i] = np.random.rand()
        guessV2[i] = np.random.rand()
    guessV1 /= np.linalg.norm(guessV1)
    guessV2 /= np.linalg.norm(guessV2)
    # Gram-Schmidt orthogonalization
    guessV2 -= np.dot(guessV2.T, guessV1) * guessV1
    guessV2 /= np.linalg.norm(guessV2)
    guessV1 = np.reshape(guessV1, (dimension, 1))
    guessV2 = np.reshape(guessV2, (dimension, 1))
    Vectors = np.hstack((guessV1, guessV2))
    norm = 1

    eigenvector = []
    eigenvalue = 0

    while norm > 0.00001:
        # project the FCI matrix onto the guess basis
        projection = np.dot(Vectors.T, np.dot(matrix, Vectors))
        # diagonalize the projection
        theta, s = np.linalg.eigh(projection)
        # compute the residue vector
        r = np.dot((np.dot(matrix,Vectors)-theta[0]*Vectors),s[:,0])
        # diagonal preconditioning
        q = -r.T / (np.diag(matrix) - theta[0] + 1E-18)
        q /= np.linalg.norm(q)
        q = np.reshape(q, (dimension, 1))
        eigenvector = np.dot(Vectors, s[:,0])
        eigenvalue = theta[0]

        Vectors = np.hstack((Vectors, q))
        # QR to orthogonalize
        Vectors = np.linalg.qr(Vectors)[0]

        norm = np.linalg.norm(r)
        
    return eigenvector, eigenvalue

# But I should try implementing my own methods...


In [6]:
vec, val = rudimentaryDavidson(hamiltonian)
print(val)

-7.839908014840962


In [ ]:
from copy import deepcopy

def singlet_direct_ci_method(h1e, h2e, norb, nelec):
    nalpha = nelec/2
    nbeta = nelec/2

    possiblestrings = list(itertools.combinations(range(norb), nalpha))
    possiblestringsarray = [list(x) for x in possiblestrings]
    numb_alpha_dets = len(possiblestrings)

    def nswaps(det):
        swaps = 0
        thisdet = deepcopy(det)
        sorteddet = sorted(thisdet)
        issorted = False
        while not issorted:
            if thisdet == sorteddet:
                issorted = True
            for i in range(len(thisdet)):
                if thisdet[i] > thisdet[i+1]:
                    deti = thisdet[i]
                    detnext = thisdet[i+1]
                    thisdet[i] = detnext
                    thisdet[i+1] = deti
                    swaps += 1
        return swaps

    def f(input_index, mu, nu):
        det = deepcopy(possiblestringsarray[input_index])
        if nu not in det:
            return None
        else:
            det.remove(nu)
            if mu in det:
                return None
            else:
                det.append(mu)
                det = sorted(det)
                det = tuple(det)
                output_index = possiblestrings.index(det)
                return output_index
    
    def sigma(input_index, mu, nu):
        det = deepcopy(possiblestringsarray[input_index])
        if nu not in det:
            return None
        else:
            det.remove(nu)
            if mu in det:
                return None
            else:
                det.append(mu)
                swaps = nswaps(det)
                return swaps
    

    hamiltonian = np.zeros(len(possiblestrings)**2, len(possiblestrings)**2)

    # This is defined for spin-free; but now we need separate for alpha vs. for beta.
        
    def big_sigma(input_index, mu, nu):
        alpha_index =  input_index % numb_alpha_dets
        beta_index = (input_index - alpha_index)/numb_alpha_dets

        if mu < norb and nu < norb:
            #we act only on alpha
            return sigma(alpha_index, mu, nu)


        elif mu >= norb and nu >= norb:
            # do something
            return sigma(beta_index, mu % norb, nu $ norb)

        else:
            return None


    def big_f(input_index, mu, nu):

        
        return None

            
            


    

In [17]:
possiblestrings = list(itertools.combinations(range(6), 3))
print(len(possiblestrings))


20


In [8]:
print(list(itertools.combinations(range(6),3)))
thislist = list(itertools.combinations(range(6),3))
thisarray = [list(x) for x in thislist]
print(thisarray)

det5 = thislist[5]
print(det5)
list5 = list(det5)
print(list5)
list5.remove(4)
print(list5)

newdet5 = thisarray[5]
print(newdet5)
newdet5.remove(4)
print(newdet5)
newdet5.append(1)
print(newdet5)
newdet5 = sorted(newdet5)
print(newdet5)

# need a method to reorder newdet5
# kendall-tau distance = bubble-sort distance.

[(0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 2, 3), (0, 2, 4), (0, 2, 5), (0, 3, 4), (0, 3, 5), (0, 4, 5), (1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 3, 4), (1, 3, 5), (1, 4, 5), (2, 3, 4), (2, 3, 5), (2, 4, 5), (3, 4, 5)]
[[0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 1, 5], [0, 2, 3], [0, 2, 4], [0, 2, 5], [0, 3, 4], [0, 3, 5], [0, 4, 5], [1, 2, 3], [1, 2, 4], [1, 2, 5], [1, 3, 4], [1, 3, 5], [1, 4, 5], [2, 3, 4], [2, 3, 5], [2, 4, 5], [3, 4, 5]]
(0, 2, 4)
[0, 2, 4]
[0, 2]
[0, 2, 4]
[0, 2]
[0, 2, 1]
[0, 1, 2]


In [11]:
def nswaps(det):
    swaps = 0
    thisdet = deepcopy(det)
    sorteddet = sorted(thisdet)
    issorted = False
    while not issorted:
        if thisdet == sorteddet:
            issorted = True
        for i in range(len(thisdet)-1):
            if thisdet[i] > thisdet[i+1]:
                deti = thisdet[i]
                detnext = thisdet[i+1]
                thisdet[i] = detnext
                thisdet[i+1] = deti
                swaps += 1
    return swaps